# 模型文件地址
    
    https://github.com/PaddlePaddle/models/tree/develop/PaddleCV/image_classification
    
    wget http://paddle-imagenet-models-name.bj.bcebos.com/ResNet50_pretrained.tar
    tar -xvf ResNet50_pretrained.tar
    
# 初始化环境

    pip install opencv-python
    pip install paddlepaddle==1.5
    
    

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import cv2
import time
import sys
import math
import numpy as np
import argparse
import functools

import paddle
import paddle.fluid as fluid

#加载自定义文件
from sdk.resnet import ResNet50
from sdk.attack_pp import FGSM


In [2]:
"""
use_gpu=False

x = fluid.layers.data(name="x", shape=[1] ,dtype='int32')
x.stop_gradient=False
y = x*3

g = fluid.backward.gradients(targets=y, inputs=[x])[0]

place = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())

result = exe.run(fluid.default_main_program(),
                 fetch_list=[y.name],
                 feed={ 'x':np.array([1]).astype('int32') })
#result = result[0][0]

print(result)
"""

'\nuse_gpu=False\n\nx = fluid.layers.data(name="x", shape=[1] ,dtype=\'int32\')\nx.stop_gradient=False\ny = x*3\n\ng = fluid.backward.gradients(targets=y, inputs=[x])[0]\n\nplace = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()\nexe = fluid.Executor(place)\nexe.run(fluid.default_startup_program())\n\nresult = exe.run(fluid.default_main_program(),\n                 fetch_list=[y.name],\n                 feed={ \'x\':np.array([1]).astype(\'int32\') })\n#result = result[0][0]\n\nprint(result)\n'

# 加载模型

In [3]:
#http://www.paddlepaddle.org.cn/documentation/docs/zh/1.5/user_guides/howto/prepare_data/index_cn.html

mean = [0.485, 0.456, 0.406] 
std = [0.229, 0.224, 0.225] 

class_dim=1000
image_shape=[3,224,224]
model_name="resnet50"
use_gpu=False
pretrained_model="ResNet50_pretrained"
TOPK=3

input_layer = fluid.layers.data(name='image', shape=image_shape, dtype='float32')
#设置为可以计算梯度
input_layer.stop_gradient=False

# model definition
model = ResNet50()
out = model.net(input=input_layer, class_dim=class_dim)
out = fluid.layers.softmax(out)

#评估模式
eval_program = fluid.default_main_program().clone(for_test=True)

fetch_list = [out.name]

place = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())

#记载模型参数
fluid.io.load_persistables(exe, pretrained_model)

# 预测原始图片

In [4]:
img_path = "mug227.png"

img = cv2.imread(img_path)
img = cv2.resize(img,(image_shape[1],image_shape[2]))

img = img[:, :, ::-1].astype('float32').transpose((2, 0, 1)) / 255
img_mean = np.array(mean).reshape((3, 1, 1))
img_std = np.array(std).reshape((3, 1, 1))
img -= img_mean
img /= img_std

print(img.shape)

result = exe.run(eval_program,
                 fetch_list=fetch_list,
                 feed={ 'image':(img,) })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:TOPK]
print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
sys.stdout.flush()

(3, 224, 224)
Test-score: [0.63326067 0.23805845 0.0749701 ], class [504 967 968]


# 调用FGSM算法 无定向攻击

In [5]:
#adv_img=FGSM()

targeted=False

softmax_out=out

img_path = "mug227.png"
step_size=128.0/256
epsilon=16.0/256

img = cv2.imread(img_path)
img = cv2.resize(img,(image_shape[1],image_shape[2]))

img = img[:, :, ::-1].astype('float32').transpose((2, 0, 1)) / 255
img_mean = np.array(mean).reshape((3, 1, 1))
img_std = np.array(std).reshape((3, 1, 1))
img -= img_mean
img /= img_std

img=img.astype('float32')
img=np.expand_dims(img, axis=0)

result = exe.run(eval_program,
                 fetch_list=fetch_list,
                 feed={ 'image':img })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:1]

label = fluid.layers.data(name="label", shape=[1] ,dtype='int64')

loss = fluid.layers.cross_entropy(input=softmax_out, label=label)

#http://www.paddlepaddle.org.cn/documentation/docs/zh/1.5/api_cn/backward_cn.html
gradients = fluid.backward.gradients(targets=loss, inputs=[input_layer])[0]

o_label=np.array(pred_label).astype('int64')
o_label=np.expand_dims(o_label, axis=0)

# 评估模式
adv_program = fluid.default_main_program().clone(for_test=False)

#计算梯度
g = exe.run(adv_program,
                 fetch_list=[gradients],
                 feed={ 'image':img,
                        'label': o_label  })
g = g[0][0]

adv=img-np.sign(g)*step_size
    
result = exe.run(eval_program,
                 fetch_list=fetch_list,
                 feed={ 'image':adv })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:TOPK]

print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
sys.stdout.flush()

Test-score: [0.3853765  0.27515155 0.03314485], class [827 507 896]
